In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import  sklearn.linear_model as linearModel
from sklearn import preprocessing
from sklearn.manifold import TSNE
import numpy as np
#import seaborn
import matplotlib.pyplot as plt
from itertools import cycle
from warnings import filterwarnings
import timeit
import pymc3 as pm
import sys
import json
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression


In [7]:
sys.path.insert(0,'../src')
import bridgeSampling as bs # it contains a method to estimate the marginal likelihood according to the Bridge Sampling approach
import utilFunctions as ut          # it has different methods to handle and plot data
import BayesianModels as bm # it has methods to build and train bayesian model (Logistic Regression and Neural Nets)
import experiments as exp
import samplebiasselection as sbs

In [22]:
number = 2
fileTrain = '/home/francisco/Escritorio/code/vsbms/data/BIASEDFATS/Train_rrlyr-'+str(number)+'.csv'
fileTest = '/home/francisco/Escritorio/code/vsbms/data/BIASEDFATS/Test_rrlyr-'+str(number)+'.csv'

# Basic Ridge Logistic Regresion (Polynomial)

In [23]:
Cvalue = 1
plot = False
basicLR_stats_acc = []
basicLR_stats_f1 = []
res = []
for k in [1, 2, 3]:
    print('kernel '+str(k))
    for components in [2, 4, 6, 8, 10, 12, 14]:
        for size in[1000, 2000, 4000]:
            print('components '+str(components))
            dataTrain = pd.read_csv(fileTrain)
            dataTest = pd.read_csv(fileTest)
            print(len(dataTrain.columns)) 
            try:
                dataTrain = dataTrain.loc[:, ~dataTrain.columns.str.contains('^Unnamed')]
                dataTrain =  dataTrain.drop(['Pred', 'Pred2', 'h', 'e', 'u','ID'], axis = 1)
                dataTrain = dataTrain.loc[:, (dataTrain != dataTrain.iloc[0]).any()]
                dataTest = dataTest[list(dataTrain.columns)]

            except:
                print('---')
            dataTrain = ut.downSampling(dataTrain)
            try:
                dataTrain = dataTrain.sample(size, random_state = 0)
            except:
                print('sample bigger that population')
            yTrain = 1*(dataTrain['label'] == 'ClassA')
            yTestHO = 1*(dataTest['label'] == 'ClassA')
            del dataTrain['label']
            del dataTest['label']

            names = dataTrain.columns
            scaler = preprocessing.StandardScaler()
            dataTrain = scaler.fit_transform(dataTrain)
            dataTrain = pd.DataFrame(dataTrain, columns=names)

            dataTest = scaler.fit_transform(dataTest)
            dataTest = pd.DataFrame(dataTest, columns=names)


            pca = PCA(n_components=components)
            pca.fit(dataTrain)
            dataTrain = pca.transform(dataTrain)
            dataTrain = pd.DataFrame(dataTrain)
            dataTrain = ut.Polinomial(dataTrain,k)


            pca.fit(dataTest)
            dataTest = pca.transform(dataTest)
            dataTest = pd.DataFrame(dataTest)
            dataTest = ut.Polinomial(dataTest,k)
            #size = dataTrain.shape[0]

            start_1 = timeit.default_timer()
            skf = StratifiedKFold(n_splits=int(10))
            skf.get_n_splits(dataTrain, yTrain)
            acc_kfold_Train = []
            f1_kfold_Train = []
            i = 0

            for train_index, test_index in skf.split(dataTrain, yTrain):
                        i = i+1
                        print('training fold ' + str(i))
                        X_train, X_test = dataTrain.iloc[train_index,:], dataTrain.iloc[test_index,:]
                        y_train, y_test = yTrain.iloc[train_index], yTrain.iloc[test_index]

                        basicLR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial',
                                                     C=Cvalue).fit(X_train, y_train)
                        predictions_1_Train = basicLR.predict(X_test)

                        accTrain = accuracy_score(y_test, predictions_1_Train, normalize=True)
                        f1Train = f1_score(y_test, predictions_1_Train, pos_label = 1)

                        if plot:
                            cm = confusion_matrix(y_test, predictions_1_Train)
                            ut.plot_confusion_matrix(cm, ['all', 'rrlyr'], type = 'train')
                        print('Accuracy train: ', accTrain)
                        #print('Accuracy f1 Train: ', f1Train)
                        basicLR_stats_acc.append(accTrain)
                        basicLR_stats_f1.append(f1Train)
            accTrain = np.mean(basicLR_stats_acc)
            f1Train =np.mean(basicLR_stats_f1)
            print('----------------------------------------------------------')
            print('Summary Training')
            print('Mean Accuracy train: ', accTrain)
            print('Mean f1 Train: ', f1Train)
            stop_1 = timeit.default_timer()
            time_CV = stop_1 - start_1
            basicLR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial', C=Cvalue).fit(dataTrain, yTrain)
            predictions_1_Test = basicLR.predict(dataTest)
            print(predictions_1_Test.shape)
            print(yTestHO.shape)
            print('----------------------------------------------------------')
            print('Summary Testing')
            accTest = accuracy_score(yTestHO, predictions_1_Test, normalize=True)
            f1Test = f1_score(yTestHO, predictions_1_Test,pos_label = 1)
            if plot: 
                cm = confusion_matrix(yTestHO, predictions_1_Test)
                ut.plot_confusion_matrix(cm, ['all', 'rrlyr'], type = 'train')

            print(' Accuracy test: ', accTest)
            print(' f1 test: ', f1Test)
            print('----------------------------------------------------------')

            res.append([k, components, size,  accTrain, accTest, f1Train, f1Test])
            pd.DataFrame(res).to_csv(str(number)+'-BasicCV0107C'+str(Cvalue)+'.csv')

kernel 1
components 2
71
training fold 1
Accuracy train:  0.92
training fold 2
Accuracy train:  0.91
training fold 3
Accuracy train:  0.94
training fold 4
Accuracy train:  0.9
training fold 5
Accuracy train:  0.9
training fold 6
Accuracy train:  0.95
training fold 7
Accuracy train:  0.95
training fold 8
Accuracy train:  0.92
training fold 9
Accuracy train:  0.91
training fold 10
Accuracy train:  0.92
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.922
Mean f1 Train:  0.9278270569253533
(17339,)
(17339,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.5970355845204453
 f1 test:  0.6115959753182499
----------------------------------------------------------
components 2
71
training fold 1
Accuracy train:  0.895
training fold 2
Accuracy train:  0.915
training fold 3
Accuracy train:  0.89
training fold 4
Accuracy train:  0.925
training fold 5
Accuracy train:  0.895
training fold 6
Accuracy trai

71
training fold 1
Accuracy train:  0.955
training fold 2
Accuracy train:  0.9675
training fold 3
Accuracy train:  0.9625
training fold 4
Accuracy train:  0.955
training fold 5
Accuracy train:  0.9625
training fold 6
Accuracy train:  0.975
training fold 7
Accuracy train:  0.9625
training fold 8
Accuracy train:  0.9425
training fold 9
Accuracy train:  0.975
training fold 10
Accuracy train:  0.9475
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9464791666666668
Mean f1 Train:  0.948862715941388
(17339,)
(17339,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.6043601130399677
 f1 test:  0.5870952208980379
----------------------------------------------------------
components 10
71
training fold 1
Accuracy train:  0.96
training fold 2
Accuracy train:  0.95
training fold 3
Accuracy train:  0.98
training fold 4
Accuracy train:  0.97
training fold 5
Accuracy train:  0.94
training fold 6
Accuracy

71
training fold 1
Accuracy train:  0.925
training fold 2
Accuracy train:  0.94
training fold 3
Accuracy train:  0.905
training fold 4
Accuracy train:  0.935
training fold 5
Accuracy train:  0.9
training fold 6
Accuracy train:  0.945
training fold 7
Accuracy train:  0.93
training fold 8
Accuracy train:  0.93
training fold 9
Accuracy train:  0.93
training fold 10
Accuracy train:  0.93
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9550760869565218
Mean f1 Train:  0.957308526673801
(17339,)
(17339,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.5883268931310918
 f1 test:  0.5571410845017992
----------------------------------------------------------
components 2
71
training fold 1
Accuracy train:  0.9325
training fold 2
Accuracy train:  0.935
training fold 3
Accuracy train:  0.915
training fold 4
Accuracy train:  0.925
training fold 5
Accuracy train:  0.93
training fold 6
Accuracy train:  

71
training fold 1
Accuracy train:  0.94
training fold 2
Accuracy train:  0.97
training fold 3
Accuracy train:  0.98
training fold 4
Accuracy train:  0.98
training fold 5
Accuracy train:  0.95
training fold 6
Accuracy train:  0.99
training fold 7
Accuracy train:  0.99
training fold 8
Accuracy train:  0.98
training fold 9
Accuracy train:  0.98
training fold 10
Accuracy train:  0.98
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9587132352941176
Mean f1 Train:  0.9607990180808185
(17339,)
(17339,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.6529211603898726
 f1 test:  0.6719720920091573
----------------------------------------------------------
components 10
71
training fold 1
Accuracy train:  0.96
training fold 2
Accuracy train:  0.98
training fold 3
Accuracy train:  0.97
training fold 4
Accuracy train:  0.985
training fold 5
Accuracy train:  0.975
training fold 6
Accuracy train:  0.98

71
training fold 1
Accuracy train:  0.9275
training fold 2
Accuracy train:  0.925
training fold 3
Accuracy train:  0.9175
training fold 4
Accuracy train:  0.9275
training fold 5
Accuracy train:  0.93
training fold 6
Accuracy train:  0.935
training fold 7
Accuracy train:  0.93
training fold 8
Accuracy train:  0.92
training fold 9
Accuracy train:  0.935
training fold 10
Accuracy train:  0.94
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9593722222222223
Mean f1 Train:  0.9614841208092051
(17339,)
(17339,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.5880961993194532
 f1 test:  0.5446895320668111
----------------------------------------------------------
components 4
71
training fold 1
Accuracy train:  0.96
training fold 2
Accuracy train:  0.97
training fold 3
Accuracy train:  0.98
training fold 4
Accuracy train:  0.96
training fold 5
Accuracy train:  0.95
training fold 6
Accuracy train:

71
training fold 1
Accuracy train:  0.97
training fold 2
Accuracy train:  0.965
training fold 3
Accuracy train:  0.975
training fold 4
Accuracy train:  0.985
training fold 5
Accuracy train:  0.965
training fold 6
Accuracy train:  0.985
training fold 7
Accuracy train:  0.97
training fold 8
Accuracy train:  0.975
training fold 9
Accuracy train:  0.97
training fold 10
Accuracy train:  0.97
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9616875000000001
Mean f1 Train:  0.963699945392379
(17339,)
(17339,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.6326201049656843
 f1 test:  0.6392978482446205
----------------------------------------------------------
components 10
71
training fold 1
Accuracy train:  0.9725
training fold 2
Accuracy train:  0.98
training fold 3
Accuracy train:  0.98
training fold 4
Accuracy train:  0.965
training fold 5
Accuracy train:  0.975
training fold 6
Accuracy train

In [ ]:
Cvalue = 100
plot = False
basicLR_stats_acc = []
basicLR_stats_f1 = []
res = []
for k in [1, 2, 3]:
    print('kernel '+str(k))
    for components in [2, 4, 6, 8, 10, 12]:
        for size in[1000, 2000, 4000]:
            print('components '+str(components))
            dataTrain = pd.read_csv(fileTrain)
            dataTest = pd.read_csv(fileTest)
            print(len(dataTrain.columns)) 
            try:
                dataTrain = dataTrain.loc[:, ~dataTrain.columns.str.contains('^Unnamed')]
                dataTrain =  dataTrain.drop(['Pred', 'Pred2', 'h', 'e', 'u','ID'], axis = 1)
                dataTrain = dataTrain.loc[:, (dataTrain != dataTrain.iloc[0]).any()]
                dataTest = dataTest[list(dataTrain.columns)]

            except:
                print('---')
            dataTrain = ut.downSampling(dataTrain)
            try:
                dataTrain = dataTrain.sample(size, random_state = 0)
            except:
                print('sample bigger that population')
            yTrain = 1*(dataTrain['label'] == 'ClassA')
            yTestHO = 1*(dataTest['label'] == 'ClassA')
            del dataTrain['label']
            del dataTest['label']

            names = dataTrain.columns
            scaler = preprocessing.StandardScaler()
            dataTrain = scaler.fit_transform(dataTrain)
            dataTrain = pd.DataFrame(dataTrain, columns=names)

            dataTest = scaler.fit_transform(dataTest)
            dataTest = pd.DataFrame(dataTest, columns=names)


            pca = PCA(n_components=components)
            pca.fit(dataTrain)
            dataTrain = pca.transform(dataTrain)
            dataTrain = pd.DataFrame(dataTrain)
            dataTrain = ut.Polinomial(dataTrain,k)


            pca.fit(dataTest)
            dataTest = pca.transform(dataTest)
            dataTest = pd.DataFrame(dataTest)
            dataTest = ut.Polinomial(dataTest,k)
            #size = dataTrain.shape[0]

            start_1 = timeit.default_timer()
            skf = StratifiedKFold(n_splits=int(10))
            skf.get_n_splits(dataTrain, yTrain)
            acc_kfold_Train = []
            f1_kfold_Train = []
            i = 0

            for train_index, test_index in skf.split(dataTrain, yTrain):
                        i = i+1
                        print('training fold ' + str(i))
                        X_train, X_test = dataTrain.iloc[train_index,:], dataTrain.iloc[test_index,:]
                        y_train, y_test = yTrain.iloc[train_index], yTrain.iloc[test_index]

                        basicLR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial',
                                                     C=Cvalue).fit(X_train, y_train)
                        predictions_1_Train = basicLR.predict(X_test)

                        accTrain = accuracy_score(y_test, predictions_1_Train, normalize=True)
                        f1Train = f1_score(y_test, predictions_1_Train, pos_label = 1)

                        if plot:
                            cm = confusion_matrix(y_test, predictions_1_Train)
                            ut.plot_confusion_matrix(cm, ['all', 'rrlyr'], type = 'train')
                        print('Accuracy train: ', accTrain)
                        #print('Accuracy f1 Train: ', f1Train)
                        basicLR_stats_acc.append(accTrain)
                        basicLR_stats_f1.append(f1Train)
            accTrain = np.mean(basicLR_stats_acc)
            f1Train =np.mean(basicLR_stats_f1)
            print('----------------------------------------------------------')
            print('Summary Training')
            print('Mean Accuracy train: ', accTrain)
            print('Mean f1 Train: ', f1Train)
            stop_1 = timeit.default_timer()
            time_CV = stop_1 - start_1
            basicLR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial', C=Cvalue).fit(dataTrain, yTrain)
            predictions_1_Test = basicLR.predict(dataTest)
            print(predictions_1_Test.shape)
            print(yTestHO.shape)
            print('----------------------------------------------------------')
            print('Summary Testing')
            accTest = accuracy_score(yTestHO, predictions_1_Test, normalize=True)
            f1Test = f1_score(yTestHO, predictions_1_Test,pos_label = 1)
            if plot: 
                cm = confusion_matrix(yTestHO, predictions_1_Test)
                ut.plot_confusion_matrix(cm, ['all', 'rrlyr'], type = 'train')

            print(' Accuracy test: ', accTest)
            print(' f1 test: ', f1Test)
            print('----------------------------------------------------------')

            res.append([k, components, size,  accTrain, accTest, f1Train, f1Test])
            pd.DataFrame(res).to_csv(str(number)+'-BasicCV0707C'+str(Cvalue)+'.csv')

kernel 1
components 2
71
training fold 1
Accuracy train:  0.92
training fold 2
Accuracy train:  0.91
training fold 3
Accuracy train:  0.94
training fold 4
Accuracy train:  0.9
training fold 5
Accuracy train:  0.9
training fold 6
Accuracy train:  0.95
training fold 7
Accuracy train:  0.95
training fold 8
Accuracy train:  0.92
training fold 9
Accuracy train:  0.91
training fold 10
Accuracy train:  0.92
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.922
Mean f1 Train:  0.9278270569253533
(17339,)
(17339,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.5971509314262645
 f1 test:  0.6117503196042465
----------------------------------------------------------
components 2
71
training fold 1
Accuracy train:  0.895
training fold 2
Accuracy train:  0.915
training fold 3
Accuracy train:  0.885
training fold 4
Accuracy train:  0.925
training fold 5
Accuracy train:  0.895
training fold 6
Accuracy tra

71
training fold 1
Accuracy train:  0.955
training fold 2
Accuracy train:  0.9675
training fold 3
Accuracy train:  0.9625
training fold 4
Accuracy train:  0.955
training fold 5
Accuracy train:  0.9625
training fold 6
Accuracy train:  0.975
training fold 7
Accuracy train:  0.9625
training fold 8
Accuracy train:  0.9425
training fold 9
Accuracy train:  0.975
training fold 10
Accuracy train:  0.9475
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9461875
Mean f1 Train:  0.9485799872836187
(17339,)
(17339,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.6044177864928774
 f1 test:  0.5856840833585021
----------------------------------------------------------
components 10
71
training fold 1
Accuracy train:  0.96
training fold 2
Accuracy train:  0.95
training fold 3
Accuracy train:  0.98
training fold 4
Accuracy train:  0.97
training fold 5
Accuracy train:  0.94
training fold 6
Accuracy train: 

71
training fold 1
Accuracy train:  0.96
training fold 2
Accuracy train:  0.96
training fold 3
Accuracy train:  0.965
training fold 4
Accuracy train:  0.985
training fold 5
Accuracy train:  0.955
training fold 6
Accuracy train:  0.98
training fold 7
Accuracy train:  0.95
training fold 8
Accuracy train:  0.95
training fold 9
Accuracy train:  0.975
training fold 10
Accuracy train:  0.98
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.952608695652174
Mean f1 Train:  0.9549691617746809
(17339,)
(17339,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.6434050406597843
 f1 test:  0.6648598840045532
----------------------------------------------------------
components 4
71
training fold 1
Accuracy train:  0.965
training fold 2
Accuracy train:  0.9725
training fold 3
Accuracy train:  0.965
training fold 4
Accuracy train:  0.9575
training fold 5
Accuracy train:  0.975
training fold 6
Accuracy train

71
training fold 1
Accuracy train:  0.93
training fold 2
Accuracy train:  0.96
training fold 3
Accuracy train:  0.97
training fold 4
Accuracy train:  0.98
training fold 5
Accuracy train:  0.95
training fold 6
Accuracy train:  0.99
training fold 7
Accuracy train:  0.99
training fold 8
Accuracy train:  0.97
training fold 9
Accuracy train:  0.98
training fold 10
Accuracy train:  0.98
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9585367647058822
Mean f1 Train:  0.960628262192593
(17339,)
(17339,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.6307745544725762
 f1 test:  0.613031914893617
----------------------------------------------------------
components 12
71
training fold 1
Accuracy train:  0.965
training fold 2
Accuracy train:  0.98
training fold 3
Accuracy train:  0.97
training fold 4
Accuracy train:  0.98
training fold 5
Accuracy train:  0.98
training fold 6
Accuracy train:  0.985
t

# Basic Logistic Regresion (Polynomial)

In [ ]:
plot = False
basicLR_stats_acc = []
basicLR_stats_f1 = []
res = []
for k in [1, 2, 3]:
    print('kernel '+str(k))
    for components in [2, 4, 6, 8, 10, 12]:
        for size in[1000, 2000, 4000]:
            print('components '+str(components))
            dataTrain = pd.read_csv(fileTrain)
            dataTest = pd.read_csv(fileTest)
            print(len(dataTrain.columns)) 
            try:
                dataTrain = dataTrain.loc[:, ~dataTrain.columns.str.contains('^Unnamed')]
                dataTrain =  dataTrain.drop(['Pred', 'Pred2', 'h', 'e', 'u','ID'], axis = 1)
                dataTrain = dataTrain.loc[:, (dataTrain != dataTrain.iloc[0]).any()]
                dataTest = dataTest[list(dataTrain.columns)]

            except:
                print('---')
            dataTrain = ut.downSampling(dataTrain)
            try:
                dataTrain = dataTrain.sample(size, random_state = 0)
            except:
                print('sample bigger that population')
            yTrain = 1*(dataTrain['label'] == 'ClassA')
            yTestHO = 1*(dataTest['label'] == 'ClassA')
            del dataTrain['label']
            del dataTest['label']

            names = dataTrain.columns
            scaler = preprocessing.StandardScaler()
            dataTrain = scaler.fit_transform(dataTrain)
            dataTrain = pd.DataFrame(dataTrain, columns=names)

            dataTest = scaler.fit_transform(dataTest)
            dataTest = pd.DataFrame(dataTest, columns=names)


            pca = PCA(n_components=components)
            pca.fit(dataTrain)
            dataTrain = pca.transform(dataTrain)
            dataTrain = pd.DataFrame(dataTrain)
            dataTrain = ut.Polinomial(dataTrain,k)


            pca.fit(dataTest)
            dataTest = pca.transform(dataTest)
            dataTest = pd.DataFrame(dataTest)
            dataTest = ut.Polinomial(dataTest,k)
            #size = dataTrain.shape[0]

            start_1 = timeit.default_timer()
            skf = StratifiedKFold(n_splits=int(10))
            skf.get_n_splits(dataTrain, yTrain)
            acc_kfold_Train = []
            f1_kfold_Train = []
            i = 0

            for train_index, test_index in skf.split(dataTrain, yTrain):
                        i = i+1
                        print('training fold ' + str(i))
                        X_train, X_test = dataTrain.iloc[train_index,:], dataTrain.iloc[test_index,:]
                        y_train, y_test = yTrain.iloc[train_index], yTrain.iloc[test_index]

                        basicLR = LogisticRegression(penalty='none', solver ='saga').fit(X_train, y_train)
                        predictions_1_Train = basicLR.predict(X_test)

                        accTrain = accuracy_score(y_test, predictions_1_Train, normalize=True)
                        f1Train = f1_score(y_test, predictions_1_Train, pos_label = 1)

                        if plot:
                            cm = confusion_matrix(y_test, predictions_1_Train)
                            ut.plot_confusion_matrix(cm, ['all', 'rrlyr'], type = 'train')
                        print('Accuracy train: ', accTrain)
                        #print('Accuracy f1 Train: ', f1Train)
                        basicLR_stats_acc.append(accTrain)
                        basicLR_stats_f1.append(f1Train)
            accTrain = np.mean(basicLR_stats_acc)
            f1Train =np.mean(basicLR_stats_f1)
            print('----------------------------------------------------------')
            print('Summary Training')
            print('Mean Accuracy train: ', accTrain)
            print('Mean f1 Train: ', f1Train)
            stop_1 = timeit.default_timer()
            time_CV = stop_1 - start_1
            basicLR = LogisticRegression(penalty='none', solver ='saga').fit(dataTrain, yTrain)
            predictions_1_Test = basicLR.predict(dataTest)
            print(predictions_1_Test.shape)
            print(yTestHO.shape)
            print('----------------------------------------------------------')
            print('Summary Testing')
            accTest = accuracy_score(yTestHO, predictions_1_Test, normalize=True)
            f1Test = f1_score(yTestHO, predictions_1_Test,pos_label = 1)
            if plot: 
                cm = confusion_matrix(yTestHO, predictions_1_Test)
                ut.plot_confusion_matrix(cm, ['all', 'rrlyr'], type = 'train')

            print(' Accuracy test: ', accTest)
            print(' f1 test: ', f1Test)
            print('----------------------------------------------------------')

            res.append([k, components, size,  accTrain, accTest, f1Train, f1Test])
            pd.DataFrame(res).to_csv('BasicCVAllDatawhitoutpenalty-0707'+str(number)+'.csv')